# Version Memo

1. Feature Engineering
    - Base Features From ONELUX
      <br>https://www.kaggle.com/code/leehomhuang/catboost-baseline-with-lots-features-inference
    
    - Date Related Features
    
    - Add Mouse-Tracking Related Features
    
    - #Features by Questions 
      <br>1301, 1931, 2282 + predict_proba for Prior-Questions

# Rawdata Info.

__Files__ <br><br>
1. train.csv - the training set <br>
2. test.csv - the test set <br>
3. sample_submission.csv - a sample submission file in the correct format <br>
4. train_labels.csv - correct value for all 18 questions for each session in the training set <br><br>

__Columns__ <br><br>
1. session_id - the ID of the session the event took place in <br>
2. index - the index of the event for the session <br>
3. elapsed_time - how much time has passed (in milliseconds) between the start of the session and when the event was recorded <br>
4. event_name - the name of the event type <br>
5. name - the event name (e.g. identifies whether a notebook_click is is opening or closing the notebook) <br>
6. level - what level of the game the event occurred in (0 to 22) <br>
7. page - the page number of the event (only for notebook-related events) <br>
8. room_coor_x - the coordinates of the click in reference to the in-game room (only for click events) <br>
9. room_coor_y - the coordinates of the click in reference to the in-game room (only for click events) <br>
10. screen_coor_x - the coordinates of the click in reference to the player’s screen (only for click events) <br>
11. screen_coor_y - the coordinates of the click in reference to the player’s screen (only for click events) <br>
12. hover_duration - how long (in milliseconds) the hover happened for (only for hover events) <br>
13. text - the text the player sees during this event <br>
14. qid - the fully qualified ID of the event <br>
15. room_fqid - the fully qualified ID of the room the event took place in <br>
16. text_fqid - the fully qualified ID of the <br>
17. fullscreen - whether the player is in fullscreen mode <br>
18. hq - whether the game is in high-quality <br>
19. music - whether the game music is on or off <br>
20. level_group - which group of levels - and group of questions - this row belongs to (0-4, 5-12, 13-22) <br><br>

__Question__ <br><br>
1. level 0-4 : Q1~Q3
2. level 5-12 : Q4~Q13
3. level 13-22 : Q14~Q18

# Files Info.

<br>
- init_data.pickle : 용량을 줄이기 위해 raw 데이터를 전처리한 데이터
<br><br>
- (AF)feature_true.pickle : ONELUX의 Base Features 리스트(feature 이름 리스트)
<br><br>
- (AF)trainset.pickle : 특징공학을 거친 학습용 데이터셋
<br><br>

# Import Libraries

In [2]:
import re
import numpy as np
import pandas as pd
import pickle
import polars as pl
from tqdm import tqdm

import warnings
warnings.simplefilter(action='ignore')

pd.set_option('display.max_rows', 10000)

# 0. pre-process Raw data

In [ ]:
# train_x data 불러오기 & 데이터형 변환해 용량 줄이기
dtypes = {  'session_id':'string', 
            'elapsed_time':np.int32,
            'event_name':'string',
            'name':'string',
            'level':np.uint8,
            'page':'string',
            'room_coor_x':np.float32,
            'room_coor_y':np.float32,
            'screen_coor_x':np.float32,
            'screen_coor_y':np.float32,
            'hover_duration':np.float32,
            'text':'string',
            'fqid':'string',
            'room_fqid':'string',
            'text_fqid':'string',
            'fullscreen':'string',
            'hq':'string',
            'music':'string',
            'level_group':'string'}

train_x = pd.read_csv("./data/train.csv", dtype = dtypes)
train_x.drop(columns = ["fullscreen", "hq", "music"], axis = 1, inplace = True)
train_x = train_x.sort_values(by = ['session_id', 'elapsed_time'], ascending = True)
print(train_x.shape)
train_x.head()

In [ ]:
# train_y data 불러오기 및 학습용으로 전처리
train_y = pd.read_csv("./data/train_labels.csv")
train_y["session"] = pd.to_numeric(train_y.session_id.str.split("_").str[0])
train_y["q"] = pd.to_numeric(train_y.session_id.str.split("_q").str[1])
train_y = train_y[["session", "q", "correct"]].copy().set_index("session")
train_y.head()

In [ ]:
# 전처리한 train_x & train_y를 init_data.pickle에 저장
init_data = {"train_x" : train_x, 
             "train_y" : train_y}

with open('./data/init_data.pickle', 'wb') as f:
    pickle.dump(init_data, f)

# 1. Feature Engineering

### Memo

 - 18문제의 정오답 분류 모델을 구축하기 위한 학습용 데이터셋을 만들어야.
 
 - 해당 게임은 총 3가지 part로 구분됨.
    
    (1) level 0-4 : Q1~Q3
    
    (2) level 5-12 : Q4~Q13
    
    (3) level 13-22 : Q14~Q18
    
 - 학습용 데이터를 Q1 ~ Q3 / Q4 ~ Q13 / Q14 ~ Q18 학습용으로 세가지 구성

### 절차

(1) Base Features 추출

(2) 시간 관련 변수 추출

(3) Base Features로부터 파생변수 추가(hm_features)

(4) Mouse-Tracking 관련 features 추가

(5) 결측치 많은 features 제거

(6) 최종 학습용 데이터 저장

In [25]:
with open('./data/init_data.pickle', 'rb') as f:
    init_data = pickle.load(f)
    
train_x = init_data["train_x"]
train_y = init_data["train_y"]
del init_data

In [26]:
train_x

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26296941,22100221145014656,1600,5483231,navigate_click,undefined,22,NaN,343.887291,36.701026,483.0,273.0,NaN,NaN,NaN,tunic.capitol_2.hall,NaN,13-22
26296942,22100221145014656,1601,5485166,navigate_click,undefined,22,NaN,332.696070,141.493178,545.0,221.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,13-22
26296943,22100221145014656,1602,5485917,navigate_click,undefined,22,NaN,369.912859,140.569205,611.0,217.0,NaN,NaN,NaN,tunic.capitol_2.hall,NaN,13-22
26296944,22100221145014656,1603,5486753,navigate_click,undefined,22,NaN,252.299653,123.805889,526.0,232.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,13-22


### (1) Base Features 추출

In [3]:
""" ONELUX의 Base Features 목록(feature 이름 목록) 불러오기 """

with open('./data/(AF)feature_true.pickle', 'rb') as f:
    features = pickle.load(f)    

In [6]:
features[0]

['col_idx_agg_count',
 'col_idx_filc_txt_val_that_agg_count',
 'col_idx_filc_txt_val_this_agg_count',
 'col_idx_filc_txt_val_it_agg_count',
 'col_idx_filc_txt_val_you_agg_count',
 'col_idx_filc_txt_val_find_agg_count',
 'col_idx_filc_txt_val_Found_agg_count',
 'col_idx_filc_txt_val_notebook_agg_count',
 'col_idx_filc_txt_val_Wells_agg_count',
 'col_idx_filc_txt_val_help_agg_count',
 'col_idx_filc_txt_val_need_agg_count',
 'col_idx_filc_txt_val_Ooh_agg_count',
 'col_idx_filc_txt_val_Jo_agg_count',
 'col_idx_filc_txt_val_can_agg_count',
 'col_idx_filc_txt_val_and_agg_count',
 'col_idx_filc_txt_val_is_agg_count',
 'col_idx_filc_txt_val_the_agg_count',
 'col_idx_filc_txt_val_to_agg_count',
 'col_etd_filc_txt_val_that_agg_mean',
 'col_etd_filc_txt_val_this_agg_mean',
 'col_etd_filc_txt_val_it_agg_mean',
 'col_etd_filc_txt_val_you_agg_mean',
 'col_etd_filc_txt_val_find_agg_mean',
 'col_etd_filc_txt_val_Found_agg_mean',
 'col_etd_filc_txt_val_notebook_agg_mean',
 'col_etd_filc_txt_val_Wells_a

In [9]:
print("Number of Features by Groups", [len(features[idx]) for idx in range(3)])

Number of Features by Groups [623, 964, 1131]


In [10]:
# Feature 이름 목록으로부터 feature 데이터를 생성하는 함수들
""" Functions for calculating feature values from predetermined feature names """

def remove_quotes(text):
    return re.sub(r"[`'/\\ *]", "", text)

def ext_col(col_name) :
    pattern = r"col_"
    if col_name.startswith(pattern) :
        pattern = r"col_"
        ext = col_name[len(pattern):].split("_")[0]
    else :
        ext = None
                
    return ext

def ext_agg(col_name) :
    pattern = r"_agg_(.*)"
    res = re.search(pattern, col_name)
    if res :
        ext = res.group(1)
    else : 
        ext = None
    
    return ext

def ext_fil(col_name) :
    start_index = col_name.find("fil_")
    if start_index != -1:
        start_index += len("fil_")
        end_index = col_name.find("_", start_index)
        ext = col_name[start_index:end_index]
    else:
        ext = None
    return ext

def ext_filc(col_name) :
    start_index = col_name.find("filc_")
    if start_index != -1:
        start_index += len("filc_")
        end_index = col_name.find("_", start_index)
        ext = col_name[start_index:end_index]
    else:
        ext = None
    return ext

def ext_val(col_name) :
    if "val_" in col_name:
        start_index = col_name.find("val_") + len("val_")
        end_index = col_name.find("_agg", start_index)
        ext = col_name[start_index:end_index]
        if ext == '':
            ext = None
        return ext
    else:
        return None

In [11]:
# Features for lv group2 & group3 From ONELUX
def add_features(col_name) :
    
    code_col = ext_col(col_name)
    code_agg = ext_agg(col_name)
    code_fil = ext_fil(col_name)
    code_filc = ext_filc(col_name)
    code_val = ext_val(col_name)

    col_vs = {"idx" : "index", 
              "et" : "elapsed_time",
              "en" : "event_name", 
              "nm" : "name",
              "lv" : "level",
              "pg" : "page",
              "rcx" : "room_coor_x",
              "rcy" : "room_coor_y", 
              "scx" : "screen_coor_x", 
              "scy" : "screen_coor_y", 
              "hd" : "hover_duration", 
              "txt" : "text", 
              "fq" : "fqid", 
              "rf" : "room_fqid", 
              "tfq" : "text_fqid", 
              "lvg" : "level_group", 
              "etd" : "elapsed_time_diff", 
              "md" : "move_distance",
              "vc" : "velocity"}

    if code_agg == "nunique" :
        add_col = f"pl.col('{col_vs[code_col]}').drop_nulls().n_unique().alias('{col_name}')"
    
    elif code_agg == "rep" :  
        ###############################################################################
            if "logbook_bingo_duration_agg_rep" in col_name : 
                add_col = (pl.col("elapsed_time").filter((pl.col("text") == "Here's the log book.") | (pl.col("fqid") == 'logbook.page.bingo'))
            .apply(lambda s: s.max() - s.min()).alias("logbook_bingo_duration_agg_rep"))

            if "logbook_bingo_indexCount_agg_rep" in col_name :
                 add_col = (pl.col("index").filter((pl.col("text") == "Here's the log book.") | (pl.col("fqid") == 'logbook.page.bingo'))
                    .apply(lambda s: s.max() - s.min()).alias("logbook_bingo_indexCount_agg_rep"))

            if "reader_bingo_duration_agg_rep" in col_name :
                add_col = (pl.col("elapsed_time").filter(((pl.col("event_name") == 'navigate_click') & (pl.col("fqid") == 'reader')) | 
                    (pl.col("fqid") == "reader.paper2.bingo")).apply(lambda s: s.max() - s.min()).alias("reader_bingo_duration_agg_rep"))

            if "reader_bingo_indexCount_agg_rep" in col_name :
                add_col = (pl.col("index").filter(((pl.col("event_name") == 'navigate_click') & (pl.col("fqid") == 'reader')) | 
                    (pl.col("fqid") == "reader.paper2.bingo")).apply(lambda s: s.max() - s.min()).alias("reader_bingo_indexCount_agg_rep"))

            if "journals_bingo_duration_agg_rep" in col_name :
                add_col = (pl.col("elapsed_time").filter(((pl.col("event_name") == 'navigate_click') & (pl.col("fqid") == 'journals')) | 
                (pl.col("fqid") == "journals.pic_2.bingo")).apply(lambda s: s.max() - s.min()).alias("journals_bingo_duration_agg_rep"))

            if "journals_bingo_indexCount_agg_rep" in col_name : 
                add_col = (pl.col("index").filter(((pl.col("event_name") == 'navigate_click') & (pl.col("fqid") == 'journals')) | 
                (pl.col("fqid") == "journals.pic_2.bingo")).apply(lambda s: s.max() - s.min()).alias("journals_bingo_indexCount_agg_rep"))

            if "reader_flag_duration_agg_rep" in col_name :
                add_col = (pl.col("elapsed_time").filter(((pl.col("event_name") == 'navigate_click') & (pl.col("fqid") == 'reader_flag')) | 
                (pl.col("fqid") == "tunic.library.microfiche.reader_flag.paper2.bingo"))
                .apply(lambda s: s.max() - s.min() if s.len() > 0 else 0).alias("reader_flag_duration_agg_rep"))

            if "reader_flag_indexCount_agg_rep" in col_name :
                add_col = (pl.col("index").filter(((pl.col("event_name") == 'navigate_click') & (pl.col("fqid") == 'reader_flag')) | 
                (pl.col("fqid") == "tunic.library.microfiche.reader_flag.paper2.bingo"))
                .apply(lambda s: s.max() - s.min() if s.len() > 0 else 0).alias("reader_flag_indexCount_agg_rep"))

            if "journalsFlag_bingo_duration_agg_rep" in col_name :
                add_col = (pl.col("elapsed_time").filter(((pl.col("event_name") == 'navigate_click') & (pl.col("fqid") == 'journals_flag')) | 
                (pl.col("fqid") == "journals_flag.pic_0.bingo"))
                .apply(lambda s: s.max() - s.min() if s.len() > 0 else 0).alias("journalsFlag_bingo_duration_agg_rep"))

            if "journalsFlag_bingo_indexCount_agg_rep" in col_name :
                add_col = (pl.col("index").filter(((pl.col("event_name") == 'navigate_click') & (pl.col("fqid") == 'journals_flag')) | 
                (pl.col("fqid") == "journals_flag.pic_0.bingo"))
                .apply(lambda s: s.max() - s.min() if s.len() > 0 else 0).alias("journalsFlag_bingo_indexCount_agg_rep"))

        ###############################################################################
    else :
        if code_val == None :
            add_col = f"pl.col('{col_vs[code_col]}').{code_agg}().alias('{col_name}')"

        else :
            if code_fil != None :
                add_col = f"pl.col('{col_vs[code_col]}').filter(pl.col('{col_vs[code_fil]}') == '{code_val}').{code_agg}().alias('{col_name}')"
            elif code_filc != None :
                add_col = f"pl.col('{col_vs[code_col]}').filter((pl.col('{col_vs[code_filc]}').str.contains('{code_val}'))).{code_agg}().alias('{col_name}')"
    
    return eval(add_col) if type(add_col) == str else add_col

### (2) 시간 관련 변수 추출

In [12]:
# session_id로부터 시간 feature 추출
def time_feature(df):
    df["year"] = df["session_id"].apply(lambda x: int(str(x)[:2])).astype(np.uint8)
    df["month"] = df["session_id"].apply(lambda x: int(str(x)[2:4])+1).astype(np.uint8)
    df["day"] = df["session_id"].apply(lambda x: int(str(x)[4:6])).astype(np.uint8)
    df["hour"] = df["session_id"].apply(lambda x: int(str(x)[6:8])).astype(np.uint8)
    df["minute"] = df["session_id"].apply(lambda x: int(str(x)[8:10])).astype(np.uint8)
    df["second"] = df["session_id"].apply(lambda x: int(str(x)[10:12])).astype(np.uint8)

    return df.set_index("session_id")

### (3) Base Features로부터 파생변수 추가(hm_features)

In [13]:
# base features로부터 새로운 feature 생성 
# Add other 6~7 'pluso' features at each lv groups
def hm_feature(train_dfs) :
    #############################################################################################
    """ 1st """
    feat = "col_pluso_txt_agg_count_deal_mean"
    idx_feat0 = ['col_idx_filc_txt_val_the_agg_count',
                 'col_idx_filc_txt_val_can_agg_count',
                 'col_idx_filc_txt_val_is_agg_count',
                 'col_idx_filc_txt_val_notebook_agg_count',
                 'col_idx_filc_txt_val_need_agg_count',
                 'col_idx_filc_txt_val_Ooh_agg_count',
                 'col_idx_filc_txt_val_it_agg_count',
                 'col_idx_filc_txt_val_that_agg_count',
                 'col_idx_filc_txt_val_you_agg_count',
                 'col_idx_filc_txt_val_Jo_agg_count',
                 'col_idx_filc_txt_val_to_agg_count',
                 'col_idx_filc_txt_val_Wells_agg_count',
                 'col_idx_filc_txt_val_find_agg_count',
                 'col_idx_filc_txt_val_help_agg_count',
                 'col_idx_filc_txt_val_and_agg_count',
                 'col_idx_filc_txt_val_Found_agg_count',
                 'col_idx_filc_txt_val_this_agg_count']

    idx_feat1 = ['col_idx_filc_txt_val_the_agg_count',
                 'col_idx_filc_txt_val_can_agg_count',
                 'col_idx_filc_txt_val_is_agg_count',
                 'col_idx_filc_txt_val_Oh_agg_count',
                 'col_idx_filc_txt_val_need_agg_count',
                 'col_idx_filc_txt_val_Ooh_agg_count',
                 'col_idx_filc_txt_val_it_agg_count',
                 'col_idx_filc_txt_val_that_agg_count',
                 'col_idx_filc_txt_val_you_agg_count',
                 'col_idx_filc_txt_val_Jo_agg_count',
                 'col_idx_filc_txt_val_to_agg_count',
                 'col_idx_filc_txt_val_Wells_agg_count',
                 'col_idx_filc_txt_val_found_agg_count',
                 'col_idx_filc_txt_val_find_agg_count',
                 'col_idx_filc_txt_val_help_agg_count',
                 'col_idx_filc_txt_val_and_agg_count',
                 'col_idx_filc_txt_val_this_agg_count']

    idx_feat2 = ['col_idx_filc_txt_val_the_agg_count',
                 'col_idx_filc_txt_val_can_agg_count',
                 'col_idx_filc_txt_val_is_agg_count',
                 'col_idx_filc_txt_val_Oh_agg_count',
                 'col_idx_filc_txt_val_need_agg_count',
                 'col_idx_filc_txt_val_Ooh_agg_count',
                 'col_idx_filc_txt_val_it_agg_count',
                 'col_idx_filc_txt_val_that_agg_count',
                 'col_idx_filc_txt_val_you_agg_count',
                 'col_idx_filc_txt_val_Jo_agg_count',
                 'col_idx_filc_txt_val_to_agg_count',
                 'col_idx_filc_txt_val_Wells_agg_count',
                 'col_idx_filc_txt_val_found_agg_count',
                 'col_idx_filc_txt_val_find_agg_count',
                 'col_idx_filc_txt_val_help_agg_count',
                 'col_idx_filc_txt_val_and_agg_count',
                 'col_idx_filc_txt_val_this_agg_count',
                 'col_idx_filc_txt_val_flag_agg_count']

    train_dfs[0][feat] = train_dfs[0][idx_feat0].mean(axis=1)
    train_dfs[1][feat] = train_dfs[1][idx_feat1].mean(axis=1)
    train_dfs[2][feat] = train_dfs[2][idx_feat2].mean(axis=1)
    #############################################################################################
    """ 2nd """
    feat = "col_pluso_txt_agg_count_deal_sum"
    idx_feat0 = ['col_idx_filc_txt_val_the_agg_count',
                 'col_idx_filc_txt_val_can_agg_count',
                 'col_idx_filc_txt_val_is_agg_count',
                 'col_idx_filc_txt_val_notebook_agg_count',
                 'col_idx_filc_txt_val_need_agg_count',
                 'col_idx_filc_txt_val_Ooh_agg_count',
                 'col_idx_filc_txt_val_it_agg_count',
                 'col_idx_filc_txt_val_that_agg_count',
                 'col_idx_filc_txt_val_you_agg_count',
                 'col_idx_filc_txt_val_Jo_agg_count',
                 'col_idx_filc_txt_val_to_agg_count',
                 'col_idx_filc_txt_val_Wells_agg_count',
                 'col_idx_filc_txt_val_find_agg_count',
                 'col_idx_filc_txt_val_help_agg_count',
                 'col_idx_filc_txt_val_and_agg_count',
                 'col_idx_filc_txt_val_Found_agg_count',
                 'col_idx_filc_txt_val_this_agg_count']

    idx_feat1 = ['col_idx_filc_txt_val_the_agg_count',
                 'col_idx_filc_txt_val_can_agg_count',
                 'col_idx_filc_txt_val_is_agg_count',
                 'col_idx_filc_txt_val_Oh_agg_count',
                 'col_idx_filc_txt_val_need_agg_count',
                 'col_idx_filc_txt_val_Ooh_agg_count',
                 'col_idx_filc_txt_val_it_agg_count',
                 'col_idx_filc_txt_val_that_agg_count',
                 'col_idx_filc_txt_val_you_agg_count',
                 'col_idx_filc_txt_val_Jo_agg_count',
                 'col_idx_filc_txt_val_to_agg_count',
                 'col_idx_filc_txt_val_Wells_agg_count',
                 'col_idx_filc_txt_val_found_agg_count',
                 'col_idx_filc_txt_val_find_agg_count',
                 'col_idx_filc_txt_val_help_agg_count',
                 'col_idx_filc_txt_val_and_agg_count',
                 'col_idx_filc_txt_val_this_agg_count']

    idx_feat2 = ['col_idx_filc_txt_val_the_agg_count',
                 'col_idx_filc_txt_val_can_agg_count',
                 'col_idx_filc_txt_val_is_agg_count',
                 'col_idx_filc_txt_val_Oh_agg_count',
                 'col_idx_filc_txt_val_need_agg_count',
                 'col_idx_filc_txt_val_Ooh_agg_count',
                 'col_idx_filc_txt_val_it_agg_count',
                 'col_idx_filc_txt_val_that_agg_count',
                 'col_idx_filc_txt_val_you_agg_count',
                 'col_idx_filc_txt_val_Jo_agg_count',
                 'col_idx_filc_txt_val_to_agg_count',
                 'col_idx_filc_txt_val_Wells_agg_count',
                 'col_idx_filc_txt_val_found_agg_count',
                 'col_idx_filc_txt_val_find_agg_count',
                 'col_idx_filc_txt_val_help_agg_count',
                 'col_idx_filc_txt_val_and_agg_count',
                 'col_idx_filc_txt_val_this_agg_count',
                 'col_idx_filc_txt_val_flag_agg_count']

    train_dfs[0][feat] = train_dfs[0][idx_feat0].sum(axis=1)
    train_dfs[1][feat] = train_dfs[1][idx_feat1].sum(axis=1)
    train_dfs[2][feat] = train_dfs[2][idx_feat2].sum(axis=1)
    #############################################################################################
    """ 3rd """
    feat = "col_pluso_txt_agg_count_deal_std"
    idx_feat0 = ['col_idx_filc_txt_val_the_agg_count',
                 'col_idx_filc_txt_val_can_agg_count',
                 'col_idx_filc_txt_val_is_agg_count',
                 'col_idx_filc_txt_val_notebook_agg_count',
                 'col_idx_filc_txt_val_need_agg_count',
                 'col_idx_filc_txt_val_Ooh_agg_count',
                 'col_idx_filc_txt_val_it_agg_count',
                 'col_idx_filc_txt_val_that_agg_count',
                 'col_idx_filc_txt_val_you_agg_count',
                 'col_idx_filc_txt_val_Jo_agg_count',
                 'col_idx_filc_txt_val_to_agg_count',
                 'col_idx_filc_txt_val_Wells_agg_count',
                 'col_idx_filc_txt_val_find_agg_count',
                 'col_idx_filc_txt_val_help_agg_count',
                 'col_idx_filc_txt_val_and_agg_count',
                 'col_idx_filc_txt_val_Found_agg_count',
                 'col_idx_filc_txt_val_this_agg_count']

    idx_feat1 = ['col_idx_filc_txt_val_the_agg_count',
                 'col_idx_filc_txt_val_can_agg_count',
                 'col_idx_filc_txt_val_is_agg_count',
                 'col_idx_filc_txt_val_Oh_agg_count',
                 'col_idx_filc_txt_val_need_agg_count',
                 'col_idx_filc_txt_val_Ooh_agg_count',
                 'col_idx_filc_txt_val_it_agg_count',
                 'col_idx_filc_txt_val_that_agg_count',
                 'col_idx_filc_txt_val_you_agg_count',
                 'col_idx_filc_txt_val_Jo_agg_count',
                 'col_idx_filc_txt_val_to_agg_count',
                 'col_idx_filc_txt_val_Wells_agg_count',
                 'col_idx_filc_txt_val_found_agg_count',
                 'col_idx_filc_txt_val_find_agg_count',
                 'col_idx_filc_txt_val_help_agg_count',
                 'col_idx_filc_txt_val_and_agg_count',
                 'col_idx_filc_txt_val_this_agg_count']

    idx_feat2 = ['col_idx_filc_txt_val_the_agg_count',
                 'col_idx_filc_txt_val_can_agg_count',
                 'col_idx_filc_txt_val_is_agg_count',
                 'col_idx_filc_txt_val_Oh_agg_count',
                 'col_idx_filc_txt_val_need_agg_count',
                 'col_idx_filc_txt_val_Ooh_agg_count',
                 'col_idx_filc_txt_val_it_agg_count',
                 'col_idx_filc_txt_val_that_agg_count',
                 'col_idx_filc_txt_val_you_agg_count',
                 'col_idx_filc_txt_val_Jo_agg_count',
                 'col_idx_filc_txt_val_to_agg_count',
                 'col_idx_filc_txt_val_Wells_agg_count',
                 'col_idx_filc_txt_val_found_agg_count',
                 'col_idx_filc_txt_val_find_agg_count',
                 'col_idx_filc_txt_val_help_agg_count',
                 'col_idx_filc_txt_val_and_agg_count',
                 'col_idx_filc_txt_val_this_agg_count',
                 'col_idx_filc_txt_val_flag_agg_count']

    train_dfs[0][feat] = train_dfs[0][idx_feat0].std(axis=1)
    train_dfs[1][feat] = train_dfs[1][idx_feat1].std(axis=1)
    train_dfs[2][feat] = train_dfs[2][idx_feat2].std(axis=1)
    #############################################################################################
    """ 4th """
    feat = "col_pluso_txt_val_find_agg_count_deal_sum"
    idx_feat0 = ['col_idx_filc_txt_val_find_agg_count',
                 'col_idx_filc_txt_val_Found_agg_count']

    idx_feat1 = ['col_idx_filc_txt_val_found_agg_count',
     'col_idx_filc_txt_val_find_agg_count']

    idx_feat2 = ['col_idx_filc_txt_val_found_agg_count',
     'col_idx_filc_txt_val_find_agg_count']

    train_dfs[0][feat] = train_dfs[0][idx_feat0].sum(axis=1)
    train_dfs[1][feat] = train_dfs[1][idx_feat1].sum(axis=1)
    train_dfs[2][feat] = train_dfs[2][idx_feat2].sum(axis=1)
    #############################################################################################
    """ 5th """
    feat = "col_pluso_en_val_click_agg_sum_deal_sum"
    idx_feat0 = ['col_md_fil_en_val_notebook_click_agg_sum',
                 'col_md_fil_en_val_object_click_agg_sum',
                 'col_md_fil_en_val_person_click_agg_sum',
                 'col_md_fil_en_val_observation_click_agg_sum',
                 'col_md_fil_en_val_cutscene_click_agg_sum',
                 'col_md_fil_en_val_navigate_click_agg_sum',
                 'col_md_fil_en_val_map_click_agg_sum',
                 'col_md_fil_en_val_notification_click_agg_sum']

    idx_feat1 = ['col_md_fil_en_val_notebook_click_agg_sum',
                 'col_md_fil_en_val_object_click_agg_sum',
                 'col_md_fil_en_val_person_click_agg_sum',
                 'col_md_fil_en_val_observation_click_agg_sum',
                 'col_md_fil_en_val_cutscene_click_agg_sum',
                 'col_md_fil_en_val_navigate_click_agg_sum',
                 'col_md_fil_en_val_map_click_agg_sum',
                 'col_md_fil_en_val_notification_click_agg_sum']

    idx_feat2 = ['col_md_fil_en_val_notebook_click_agg_sum',
                 'col_md_fil_en_val_object_click_agg_sum',
                 'col_md_fil_en_val_person_click_agg_sum',
                 'col_md_fil_en_val_observation_click_agg_sum',
                 'col_md_fil_en_val_cutscene_click_agg_sum',
                 'col_md_fil_en_val_navigate_click_agg_sum',
                 'col_md_fil_en_val_map_click_agg_sum',
                 'col_md_fil_en_val_notification_click_agg_sum']

    train_dfs[0][feat] = train_dfs[0][idx_feat0].sum(axis=1)
    train_dfs[1][feat] = train_dfs[1][idx_feat1].sum(axis=1)
    train_dfs[2][feat] = train_dfs[2][idx_feat2].sum(axis=1)
    #############################################################################################
    """ 6th """
    feat = "col_pluso_fq_val_flag_agg_mean_deal_mean"
    idx_feat = ['col_etd_fil_fq_val_reader_flag.paper1.next_agg_mean',
                 'col_etd_fil_fq_val_groupconvo_flag_agg_mean',
                 'col_etd_fil_fq_val_flag_girl_agg_mean',
                 'col_etd_fil_fq_val_reader_flag.paper0.next_agg_mean',
                 'col_etd_fil_fq_val_journals_flag.hub.topics_agg_mean',
                 'col_etd_fil_fq_val_reader_flag.paper0.prev_agg_mean',
                 'col_etd_fil_fq_val_tocollectionflag_agg_mean',
                 'col_etd_fil_fq_val_journals_flag.pic_1.next_agg_mean',
                 'col_etd_fil_fq_val_reader_flag.paper2.prev_agg_mean',
                 'col_etd_fil_fq_val_journals_flag_agg_mean',
                 'col_etd_fil_fq_val_tunic.flaghouse_agg_mean',
                 'col_etd_fil_fq_val_journals_flag.pic_0.next_agg_mean',
                 'col_etd_fil_fq_val_journals_flag.pic_1.bingo_agg_mean',
                 'col_etd_fil_fq_val_journals_flag.pic_0.bingo_agg_mean',
                 'col_etd_fil_fq_val_journals_flag.pic_2.next_agg_mean',
                 'col_etd_fil_fq_val_reader_flag.paper2.bingo_agg_mean',
                 'col_etd_fil_fq_val_journals_flag.hub.topics_old_agg_mean',
                 'col_etd_fil_fq_val_reader_flag.paper2.next_agg_mean',
                 'col_etd_fil_fq_val_reader_flag_agg_mean',
                 'col_etd_fil_fq_val_journals_flag.pic_2.bingo_agg_mean']

    train_dfs[2][feat] = train_dfs[2][idx_feat].mean(axis=1)
    #############################################################################################
    """ 7th """
    feat = "col_pluso_en_val_click_agg_sum_deal_prod"
    idx_feat0 = ['col_md_fil_en_val_notebook_click_agg_sum',
                 'col_md_fil_en_val_object_click_agg_sum',
                 'col_md_fil_en_val_person_click_agg_sum',
                 'col_md_fil_en_val_observation_click_agg_sum',
                 'col_md_fil_en_val_cutscene_click_agg_sum',
                 'col_md_fil_en_val_navigate_click_agg_sum',
                 'col_md_fil_en_val_map_click_agg_sum',
                 'col_md_fil_en_val_notification_click_agg_sum']

    idx_feat1 = ['col_md_fil_en_val_notebook_click_agg_sum',
                 'col_md_fil_en_val_object_click_agg_sum',
                 'col_md_fil_en_val_person_click_agg_sum',
                 'col_md_fil_en_val_observation_click_agg_sum',
                 'col_md_fil_en_val_cutscene_click_agg_sum',
                 'col_md_fil_en_val_navigate_click_agg_sum',
                 'col_md_fil_en_val_map_click_agg_sum',
                 'col_md_fil_en_val_notification_click_agg_sum']

    idx_feat2 = ['col_md_fil_en_val_notebook_click_agg_sum',
                 'col_md_fil_en_val_object_click_agg_sum',
                 'col_md_fil_en_val_person_click_agg_sum',
                 'col_md_fil_en_val_observation_click_agg_sum',
                 'col_md_fil_en_val_cutscene_click_agg_sum',
                 'col_md_fil_en_val_navigate_click_agg_sum',
                 'col_md_fil_en_val_map_click_agg_sum',
                 'col_md_fil_en_val_notification_click_agg_sum']

    train_dfs[0][feat] = train_dfs[0][idx_feat0].prod(axis=1)
    train_dfs[1][feat] = train_dfs[1][idx_feat1].prod(axis=1)
    train_dfs[2][feat] = train_dfs[2][idx_feat2].prod(axis=1)
    
    return train_dfs

### (4) Mouse-Tracking 관련 features 추가

In [14]:
# I appended 'moving distance' & 'velocity' (correctly, 'speed') features to the train_x data
columns = [
    pl.col("page").cast(pl.Float32),
    (
        (pl.col("elapsed_time") - pl.col("elapsed_time").shift(1))
        .fill_null(0)
        .clip(0, 1e9)
        .over(["session_id", "level"])
        .alias("elapsed_time_diff")
    ),
    (
        (pl.col("screen_coor_x") - pl.col("screen_coor_x").shift(1))
        .abs()
        .over(["session_id", "level"])
    ),
    (
        (pl.col("screen_coor_y") - pl.col("screen_coor_y").shift(1))
        .abs()
        .over(["session_id", "level"])
    ),
    (
        np.sqrt( ( (pl.col("screen_coor_y") - pl.col("screen_coor_y").shift(1)) )**2 
        + ( (pl.col("screen_coor_x") - pl.col("screen_coor_x").shift(1)) )**2 )
        .abs()
        .over(["session_id", "level"])
        .alias("move_distance")
    ),
    pl.col("fqid").fill_null("fqid_None"),
    pl.col("level").apply(lambda x : str(x)),
    pl.col("text_fqid").fill_null("text_fqid_None")

]
train_df = pl.from_pandas(train_x).with_columns(columns)

""" Calculate Speed feature by (moving distance / elapsed_time_diff) """
train_df = train_df.with_columns(
    (
        (pl.col("move_distance") / pl.col("elapsed_time_diff"))
        .fill_null(0)
        .clip(0, 1e9)
        .over(["session_id", "level"])
        .alias("velocity")
    )
)

train_dfs = []
for lgv in ["0-4", "5-12", "13-22"] :
    train_dfs.append(train_df.filter(pl.col("level_group") == lgv))
del train_x

In [15]:
train_dfs[2]

session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,level_group,elapsed_time_diff,move_distance,velocity
i64,i64,i64,str,str,str,f32,f64,f64,f64,f64,f64,str,str,str,str,str,i64,f64,f64
20090312431273200,512,836732,"""navigate_click...","""undefined""","""13""",null,290.153549,-204.499365,null,null,null,null,"""fqid_None""","""tunic.capitol_...","""text_fqid_None...","""13-22""",0,null,0.0
20090312431273200,513,837245,"""navigate_click...","""undefined""","""13""",null,353.805607,-210.332061,21.0,0.0,null,null,"""fqid_None""","""tunic.capitol_...","""text_fqid_None...","""13-22""",513,21.0,0.040936
20090312431273200,514,837779,"""navigate_click...","""undefined""","""13""",null,587.680024,-280.706245,108.0,44.0,null,null,"""fqid_None""","""tunic.capitol_...","""text_fqid_None...","""13-22""",534,116.619038,0.218388
20090312431273200,515,838446,"""navigate_click...","""undefined""","""13""",null,751.496869,-102.153292,43.0,124.0,null,null,"""toentry""","""tunic.capitol_...","""text_fqid_None...","""13-22""",667,131.244047,0.196768
20090312431273200,516,839629,"""map_hover""","""basic""","""13""",null,null,null,null,null,67.0,null,"""tunic.dryclean...","""tunic.capitol_...","""text_fqid_None...","""13-22""",1183,null,0.0
20090312431273200,517,840662,"""map_hover""","""basic""","""13""",null,null,null,null,null,983.0,null,"""tunic.dryclean...","""tunic.capitol_...","""text_fqid_None...","""13-22""",1033,null,0.0
20090312431273200,518,840780,"""map_hover""","""basic""","""13""",null,null,null,null,null,100.0,null,"""tunic.historic...","""tunic.capitol_...","""text_fqid_None...","""13-22""",118,null,0.0
20090312431273200,519,840830,"""map_hover""","""basic""","""13""",null,null,null,null,null,35.0,null,"""tunic.capitol_...","""tunic.capitol_...","""text_fqid_None...","""13-22""",50,null,0.0
20090312431273200,520,841212,"""map_hover""","""basic""","""13""",null,null,null,null,null,367.0,null,"""tunic.library""","""tunic.capitol_...","""text_fqid_None...","""13-22""",382,null,0.0


In [16]:
""" Create features related to speed and moving distance by changing some of the feature names in feature_true. """

# Velocity features aggregated with std, mean, max 
vfeatures = []
for idx in range(3) :
    vfeature = [feat for feat in features[idx] if ext_agg(feat) in ["std", "mean", "max"]]
    for c in ["etd_", "hd_", "rcx_", "rcy_", "scx_", "scy_", "pg_"] :
        vfeature = list(set([col.replace(f"{c}", "vc_") for col in vfeature]))
    vfeatures.append(vfeature)

# Moving distance features aggregated with sum, std, mean, max
dfeatures = []
for idx in range(3) :
    dfeature = [feat for feat in features[idx] if ext_agg(feat) in ["sum", "std", "mean", "max"]]
    for c in ["etd_", "hd_", "rcx_", "rcy_", "scx_", "scy_", "pg_"] :
        dfeature = list(set([col.replace(f"{c}", "md_") for col in dfeature]))
    dfeatures.append(dfeature)

In [17]:
feature_true = []
for idx in range(3) :
    feature_true.append(list(set(features[idx] + vfeatures[idx] + dfeatures[idx])))

In [18]:
for idx in range(3) :
    en_list = list(train_dfs[idx]["event_name"].unique())
    fq_list = list(train_dfs[idx]["fqid"].unique())
    feature_true[idx] += [f"col_idx_fil_en_val_{c}_agg_count" for c in en_list]
    feature_true[idx] += [f"col_idx_fil_fq_val_{c}_agg_count" for c in fq_list]

In [19]:
""" Make Training Dataset with Selected Features Names """ 
trainset = []
for idx in tqdm(range(3)) :
    add_cols = [add_features(feat) for feat in feature_true[idx]]
    aggs = [*add_cols]
    tmp_set = train_dfs[idx].groupby(['session_id'], maintain_order=True).agg(aggs).sort("session_id").to_pandas()
    tmp_set = time_feature(tmp_set)
    trainset.append(tmp_set)
del tmp_set
# Plus HM features
trainset = hm_feature(trainset)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:47<00:00, 15.89s/it]


### (5) 결측치 많은 features 제거

In [20]:
%%time
nulls = [ trainset[ii].isnull().sum().sort_values(ascending=False)/len(trainset[ii]) for ii in range(3)]
drops = [ list(nulls[ii][ nulls[ii] > 0.85].index) for ii in range(3) ]

print()
print(len(drops[0]), len(drops[1]), len(drops[2]))

for ii in range(3) :
    for col in tqdm(trainset[ii]) :
        if trainset[ii][col].nunique() == 1 :
            print(col)
            drops[ii].append(col)


41 166 187


  2%|█▎                                                                          | 424/23562 [00:00<00:10, 2125.41it/s]

col_md_fil_en_val_object_hover_agg_std
col_md_fil_en_val_map_hover_agg_std
col_vc_fil_en_val_checkpoint_agg_std
col_md_fil_en_val_checkpoint_agg_std


  4%|██▊                                                                         | 854/23562 [00:00<00:10, 2113.44it/s]

col_vc_fil_en_val_map_hover_agg_std
col_vc_fil_en_val_map_hover_agg_mean
col_vc_fil_en_val_object_hover_agg_mean
col_vc_fil_en_val_map_hover_agg_max


...

col_vc_fil_en_val_map_hover_agg_std
col_vc_fil_en_val_map_hover_agg_mean
col_vc_fil_en_val_object_hover_agg_mean


  8%|██████▏                                                                    | 1958/23562 [00:00<00:09, 2179.27it/s]

col_vc_fil_en_val_map_hover_agg_max
col_vc_fil_en_val_object_hover_agg_max
col_vc_fil_en_val_checkpoint_agg_mean


 11%|███████▉                                                                   | 2479/23562 [00:01<00:09, 2218.81it/s]

col_vc_fil_en_val_checkpoint_agg_max
col_vc_fil_en_val_object_hover_agg_std
CPU times: total: 2.94 s
Wall time: 2.9 s


### (6) 최종 학습용 데이터 저장

### 최종 선택된 features

-> Q1 ~ Q3 #features 1301개

-> Q4 ~ Q13 #features 1931개

-> Q14 ~ Q18 #features 2282개

In [21]:
feature_true = []
for ii in range(3) :
    feature = [c for c in trainset[ii].columns if c not in drops[ii] + ["level_group"]]
    feature_true.append(feature)
print(f"NB of Features : {len(feature_true[0])}, {len(feature_true[1])}, {len(feature_true[2])}")
train_dfs = [trainset[ii][feature_true[ii]] for ii in range(3)]

NB of Features : 1301, 1931, 2282


In [22]:
# 최종 학습용 데이터 저장 '(AF)trainset.pickle'
# Final train dataset by lv groups
with open('./data/(AF)trainset.pickle', 'wb') as f:
    pickle.dump(train_dfs, f) 